In [1]:
import pymysql.cursors
import pandas as pd
connection = pymysql.connect(host='ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com', 
                             user='read_only', password='WHC63fKyjudfSz2U', db='CAPC_APIGATEWAY',
                             cursorclass=pymysql.cursors.SSDictCursor)

In [2]:
connection

In [3]:
acc = []
for i in open('/home/trishalaswain/Documents/encDocIds','r').readlines():
    acc.append(i.rstrip('\n'))

In [4]:
len(acc)

13

In [5]:
# sql = """SELECT em.id,dm.updated_date,am.account_number as account_no,dm.id as document_id,dc.code as suggested_code,dm.name as document_name FROM
# CAPC_APIGATEWAY.account_mst am INNER JOIN
# CAPC_APIGATEWAY.encounter_mst em ON am.id = em.account_id INNER JOIN
# CAPC_APIGATEWAY.document_mst dm ON dm.account_id = am.id INNER JOIN
# CAPC_APIGATEWAY.document_code dc ON dm.id = dc.document_id
# WHERE facility_id = 333 AND dm.is_active=1 and am.is_active=1 and em.is_active=1 and
# dm.updated_date in (select max(updated_date) from document_mst where am.id=account_id) and
#  dc.code_category = 'EANDM' AND
#  am.account_number in ({}) AND
#  ORDER BY dm.updated_date,am.account_number DESC""".format(','.join(str(r) for r in acc))


In [6]:
# sql = """SELECT 
#     em.id,
#     dm.updated_date,
#     am.id,
#     GROUP_CONCAT(DISTINCT am.account_number) AS account_no,
#     dm.id AS document_id,
#     GROUP_CONCAT(DISTINCT dm.name) AS document_name,
#     GROUP_CONCAT(dc.code) AS suggested_codes
# FROM
#     CAPC_APIGATEWAY.account_mst am
#         INNER JOIN
#     CAPC_APIGATEWAY.document_mst dm ON dm.account_id = am.id
#         AND dm.is_physician_document = 1
#         AND dm.is_active = 1
#         INNER JOIN
#     CAPC_APIGATEWAY.encounter_mst em ON dm.account_id = em.account_id
#         AND dm.service_date = em.service_date
#         AND dm.physician_id = em.physician_id
#         AND em.is_active = 1
#         INNER JOIN
#     CAPC_APIGATEWAY.document_code dc ON dm.id = dc.document_id
# WHERE
#         am.facility_id = 288
#         AND am.is_active = 1
#         AND dm.id = (SELECT 
#             id
#         FROM
#             document_mst
#         WHERE
#             account_id = em.account_id
#                 AND service_date = em.service_date
#                 AND physician_id = em.physician_id
#         ORDER BY updated_date DESC
#         LIMIT 1)
#         AND dc.code_category = 'EANDM'
#         AND em.id in ({})
# GROUP BY em.id , dm.updated_date , am.id , dm.id""".format(",".join("'"+str(x)+"'" for x in acc))

In [7]:
# only pcs download by encDocIds
sql = """select dm.id,nlp_pcs_xml_result_s3_path from document_processing_detail dpd
inner join
document_mst dm ON dm.id = dpd.document_id
inner join
account_mst am ON am.id = dm.account_id
where dm.id in ({});""".format(",".join("'"+str(x)+"'" for x in acc))

In [8]:
# pcs and eandmcs_request.json download by encDocIds
# sql = """select dm.id,dpd.nlp_pcs_xml_result_s3_path,dpd.eandmcs_request_s3_path from document_processing_detail dpd
# inner join
# document_mst dm ON dm.id = dpd.document_id
# inner join
# account_mst am ON am.id = dm.account_id
# where dm.id in ({});""".format(",".join("'"+str(x)+"'" for x in acc))

In [9]:
# sql = """SELECT code,id FROM CAPC_APIGATEWAY.eandm_code where eandm_code.id in ({});"""

In [10]:
sql_query = pd.read_sql(sql,con= connection)

/home/trishalaswain/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
sql_query

,id,nlp_pcs_xml_result_s3_path
0,29454303,ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230201/6167...
1,29454314,ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230201/6167...
2,29454396,ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230201/6167...
3,29454542,ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230201/6167...
4,29627074,ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230202/6198...
5,29808894,ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230203/6230...
6,29809058,ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230203/6230...
7,29809089,ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230203/6230...
8,29809523,ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230203/6230...
9,29809780,ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230203/6230...


In [12]:
# cmd1 = []
# for pcs_path,request_path,enc_id in zip(sql_query.nlp_pcs_xml_result_s3_path,sql_query.eandmcs_request_s3_path,sql_query.id):
#     # cmd1.append("aws s3 cp s3://ezdi-production-bucket/"+str(pcs_path)+"    "+str(enc_id)+"/"+str(enc_id) +"_pcs.xml")
#     cmd1.append("aws s3 cp s3://ezdi-production-bucket/"+str(pcs_path)+"    "+str(enc_id) +"_pcs.xml")

#     # cmd1.append("aws s3 cp s3://ezdi-production-bucket/"+str(request_path)+"    "+str(enc_id)+"/"+ str(enc_id)+"_request.json")

In [13]:
cmd1 = []
for pcs_path,enc_id in zip(sql_query.nlp_pcs_xml_result_s3_path,sql_query.id):
    # cmd1.append("aws s3 cp s3://ezdi-production-bucket/"+str(pcs_path)+"    "+str(enc_id)+"/"+str(enc_id) +"_pcs.xml")
    pcs_path = pcs_path.replace("'","\'")
    pcs_path = pcs_path.replace(" ","\ ")
    # pcs_path = pcs_path.replace("'","\'")
    cmd1.append("aws s3 cp s3://ezdi-production-bucket/"+str(pcs_path)+"    "+str(enc_id) +"_pcs.xml")

    # cmd1.append("aws s3 cp s3://ezdi-production-bucket/"+str(request_path)+"    "+str(enc_id)+"/"+ str(enc_id)+"_request.json")

In [14]:
cmd1

['aws s3 cp s3://ezdi-production-bucket/ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230201/6167888__LCH_1613771_TUBERA_MICHAEL_2023-02-01_1352_126609124_20230206152054.054_result.xml    29454303_pcs.xml',
 'aws s3 cp s3://ezdi-production-bucket/ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230201/6167890__LCH_1613848_SHIPLEE_CATHERINE_2023-02-01_1447_126612379_20230206152100.000_result.xml    29454314_pcs.xml',
 'aws s3 cp s3://ezdi-production-bucket/ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230201/6167919__LCH_1613451_TOLENTINO_MARIEL\\ CHAELI\\ ARICIA\\ CO_2023-02-01_0556_126587767_20230206152155.055_result.xml    29454396_pcs.xml',
 'aws s3 cp s3://ezdi-production-bucket/ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230201/6167932__LCH_1613948_OBADO_SOLIDAD_2023-02-01_1725_126619978_20230206152211.011_result.xml    29454542_pcs.xml',
 'aws s3 cp s3://ezdi-production-bucket/ezcapc/PH/data/ERLCH/nlp/pcs-xml/20230202/6198901__LCH_1614437_BROWN_DUKE\\ KEKOA_2023-02-02_1806_126657772_20230207150958.058_result.xml    29627074

In [15]:
with open(r'/home/trishalaswain/Downloads/common-server/sh-files/testing.sh', 'w') as fp:
    for item in cmd1:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done
